In [146]:

#PROGRAMA QUE IMPLEMENTA LA SIMULACIÓN DE UN ROBOT DE LIMPIEZA REACTIVO
#PROGRAMADORES: LEÓN EMILIANO GARCÍA PÉREZ, KEVIN FERNANDO MEZA NARVÁEZ, CARLA MORALES LÓPEZ, JOSÉ ANGEL TOBÓN SALAZAR
#FECHA DE ENTREGA: LUNES 14 DE NOVIEMBRE DE 2022


In [11]:

#IMPORTACIÓN PARA EL DISEÑO DEL MODELO
import agentpy as ap

#IMPORTACIÓN PARA LA VISUALIZACIÓN
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#IMPORTACIÓN PARA EL DESARROLLO DE LA LÓGICA
import random


#DEFINICIÓN DE LA CLASE "ROOM-MODEL" QUE HEREDA DE AP.MODEL

class RoomModel(ap.Model):
    
    #DEFINICIÓN DEL MÉTODO "SETUP" EL CUAL INICIALIZA EL MODELO
    def setup(self):
        
        #CREACIÓN DE LOS AGENTES "DIRT"
        nDirts = int(self.p.dirtDensity * (self.p.height*self.p.length))
        dirts = self.agents = ap.AgentList(self,nDirts)
        
        #CREACIÓN DE LOS AGENTES "VACUUM"
        vacuums = self.cleanerAgents = ap.AgentList(self,self.p.howManyAgents)
        
        #POSICIONAMIENTO DE "DIRTS" EN EL GRID
        self.room = ap.Grid(self,[self.p.height, self.p.length], track_empty=True)
        self.room.add_agents(dirts, random=True, empty=True)
        
        #POSICIONAMIENTO DE "VACUUMS" EN EL GRID
        tuplePositions = [(1,1) for x in range (len(self.cleanerAgents))]
        self.room.add_agents(vacuums, positions = tuplePositions, empty = True)
        
        #INICIALIZACIÓN DE LAS VARIABLES DINÁMICAS DE "DIRT"
        #ESTADOS -> 0:SUCIO 1:LIMPIANDO 2:LIMPIO
        #ESTADOS -> 0:dirtUnclean 1:dirtCleaning 2:dirtCleaned
        
        self.agents.condition = 0
        
        #INICIALIZACIÓN DE LAS VARIABLES DINÁMICAS DE "VACUUM"
        #ESTADO -> 3:EXISTIENDO
        
        self.cleanerAgents.condition = 3
        self.cleanerAgents.x = 1
        self.cleanerAgents.y = 1
        self.cleanerAgents.moves = 0
    
    #DEFINICIÓN DEL MÉTODO "STEP" EL CUAL ACTUALIZA EL MODELO
    def step(self):
        
        validMoves = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,1),(1,0)]
        
        #SELECCIÓN DE SUCIEDADES
        dirtUnclean = self.agents.select(self.agents.condition == 0)
        
        for cleaner in self.cleanerAgents:
            
            steeping = self.room.agents[cleaner.x,cleaner.y]
            
            if steeping.condition == 0:
                steeping.condition = 2
             
            randomMove = random.choice(validMoves)
            newX , newY = randomMove
            
            cleaner.x = cleaner.x + newX
            cleaner.y = cleaner.y + newY
            
            if cleaner.x < 0 or cleaner.x > self.p.length and cleaner.y < 0 or cleaner.y > self.p.height:
                cleaner.moves = cleaner.moves + 0
            else:
                cleaner.moves = cleaner.moves + 1
            
            if cleaner.x > self.p.length:
                cleaner.x = self.p.length
            elif cleaner.x < 0:
                cleaner.x = 0
                
            if cleaner.y > self.p.height:
                cleaner.y = self.p.height
            elif cleaner.y < 0:
                cleaner.y = 0
                
            self.room.move_by(cleaner,randomMove)
            
            self.room.move_by(cleaner,(0,0))
            cleaner.condition = 3
        
        
        #SI NO HAY MÁS SUCIEDAD QUE LIMPIAR, ENTONCES FINALIZA LA SIMULACIÓN
        if len(dirtUnclean) == 0:
            self.stop()
        
    #DEFINICIÓN DEL MÉTODO "END", EL CUAL REALIZA LAS MEDICIONES AL FINAL DE LA SIMULACIÓN
    def end(self):
        
        dirtCleaned = len(self.agents.select(self.agents.condition == 2))
        self.report('Porcentaje de Suciedad Limpiada',dirtCleaned / len(self.agents))
        self.report('Steps realizados',model.t)
        #self.report('Movimientos de cada Agente Limpiador', self.cleanerAgents.moves)



#DEFINICIÓN DE PARÁMETROS

parameters = {
    'dirtDensity': 0.85,
    'length' : 50,
    'height' : 50,
    'howManyAgents' : 20,
    'steps' : 100
}



#CREACIÓN DE LA ANIMACIÓN

def animation_plot(model, ax):
    attr_grid = model.room.attr_grid('condition')
    color_dict = {0:'#5B1F00', 1:'#874421', 2:'#E6B68B', 3:'#03BB85', None:'#FFFFFF'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of the cleaning of a room\n"
                f"Time-step: {model.t}, Dirt left: "
                f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = RoomModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=10))
